In [12]:
import tensorflow as tf
from tensorflow import keras
from time import time
import numpy as np
from tensorflow.python.keras.callbacks import TensorBoard

In [15]:
## https://tensorboard.dev/experiment/UaQxHoFoR8q3li7GL4Sr9w/

**Program without param turnings**

Сделал отдельную программу для себя, чтобы разобраться, как работает Tensorboard

In [ ]:
(train_img, train_lab), (test_img, test_lab) = keras.datasets.fashion_mnist.load_data()

train_img = train_img/255.0
test_img = test_img/255.0

model = keras.Sequential([keras.layers.Flatten(input_shape = (28, 28)), 
                            keras.layers.Dense(128, activation = tf.nn.relu), 
                            keras.layers.Dense(10, activation = tf.nn.softmax)])

tb = TensorBoard(log_dir = "logs/{}".format(time()))

model.compile(optimizer = tf.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_img, train_lab, epochs = 10, callbacks = [tb])

test_loss, test_acc = model.evaluate(test_img, test_lab)

print('accuracy', test_acc)

4431872/4422102 [==============================] - 0s 0us/step
Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5031 - accuracy: 0.8227
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3776 - accuracy: 0.8629
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3400 - accuracy: 0.8759
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3158 - accuracy: 0.8837
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2960 - accuracy: 0.8906
Epoch 6/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2833 - accuracy: 0.8957
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2693 - accuracy: 0.9002
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2586 - accuracy: 0.9036
Epoch 9/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2470 - accuracy: 0.9084
Epoch 10/10
313/313

In [ ]:
'''
create 'log' folder on the Desktop, 
pass events.out and write command in terminal: 
python3 -m tensorboard.main --logdir=log
'''

**Program with turnings num_units, dropout, optimizer**

In [23]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

def train_test_model(hparams):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
    ])
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )

    model.fit(x_train, y_train, epochs=1) 
    _, accuracy = model.evaluate(x_test, y_test)
    return accuracy

def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
313/313 [==============================] - 1s 1ms/step - loss: 0.4842 - accuracy: 0.8296
--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
313/313 [==============================] - 1s 1ms/step - loss: 0.6861 - accuracy: 0.7544
--- Starting trial: run-2
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'adam'}
313/313 [==============================] - 1s 1ms/step - loss: 0.5212 - accuracy: 0.8132
--- Starting trial: run-3
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'sgd'}
313/313 [==============================] - 1s 1ms/step - loss: 0.6701 - accuracy: 0.7681
--- Starting trial: run-4
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'adam'}
313/313 [==============================] - 1s 1ms/step - loss: 0.4735 - accuracy: 0.8318
--- Starting trial: run-5
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'sgd'}
313/313 [==============================] - 1s 1ms/step - loss: 0.6056 - 

Набор параметров указан в таблице hparams_table.csv, ниже представлена модель с оптимальными параметрами и рассчет ошибок на каждом эпохе на train выборке

In [29]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

METRIC_ACCURACY = 'accuracy'

model = tf.keras.models.Sequential([
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(32.0, activation=tf.nn.relu),
tf.keras.layers.Dropout(0.1),
tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

losses = []

for i in range(10):
  model.fit(x_train, y_train, epochs=1) 
  _, accuracy = model.evaluate(x_train, y_train)
  losses.append(_)

1875/1875 [==============================] - 3s 1ms/step - loss: 0.3018 - accuracy: 0.8875


In [30]:
losses

[0.42206519842147827,
 0.38051554560661316,
 0.35546502470970154,
 0.33573856949806213,
 0.3371143043041229,
 0.3231034278869629,
 0.3184434771537781,
 0.3000716269016266,
 0.2955131232738495,
 0.3018425703048706]

In [32]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3768 - accuracy: 0.8650


[0.3767769932746887, 0.8650000095367432]

In [46]:
# pred and real values for first 20 test images

for i in range(20):
  pred = model.predict(x_test)[i]
  print(list(pred).index(max(pred)), y_test[i])

9 9
2 2
1 1
1 1
6 6
1 1
4 4
6 6
5 5
7 7
4 4
5 5
5 7
3 3
4 4
1 1
2 2
2 4
8 8
0 0
